In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy 
import tqdm 
import matplotlib.pyplot as plt

2021-11-18 15:35:35.731884: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
path_to_file = '../data/folclore-web/clean_dataset.txt'
#path_to_file = '../data/chamame-web/clean_dataset.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))
print('El largo del dataset es de {} caracteres y contiene {} caracteres unicos'.format(len(text), len(set(vocab))))

El largo del dataset es de 1933003 caracteres y contiene 37 caracteres unicos


# Procesamiento del texto


Primero se convierte el texto en tokens (separo en caracteres), despues uso la capa StringLookup para convertir cada caracter en un numero. 

In [4]:
chars = tf.strings.unicode_split(text, input_encoding='UTF-8') # divido en caracteres
# armo una capa para ir de ids a chars, y otra para ir de chars a ids
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

2021-11-18 15:35:37.701570: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-18 15:35:37.746541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 15:35:37.746719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7715GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-11-18 15:35:37.746735: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-18 15:35:37.749398: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-18 15:35:37.749425: I tensorflow/stream_executo

In [5]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [6]:
# Probando resultados con baches
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

In [7]:
for i in dataset.take(1):
    print(i)

2021-11-18 15:35:39.451261: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-18 15:35:39.470265: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


(<tf.Tensor: shape=(64, 200), dtype=int64, numpy=
array([[32, 18, 26, ..., 26, 24, 26],
       [ 2,  1, 28, ..., 25, 15, 26],
       [ 1, 31, 29, ..., 30, 12, 21],
       ...,
       [20, 26,  2, ...,  2, 27, 16],
       [23, 12,  2, ..., 18, 26,  1],
       [26,  2, 28, ..., 36,  2, 32]])>, <tf.Tensor: shape=(64, 200), dtype=int64, numpy=
array([[18, 26,  2, ..., 24, 26,  2],
       [ 1, 28, 32, ..., 15, 26,  2],
       [31, 29, 16, ..., 12, 21, 16],
       ...,
       [26,  2,  2, ..., 27, 16, 14],
       [12,  2, 30, ..., 26,  1,  1],
       [ 2, 28, 32, ...,  2, 32, 25]])>)


# Modelo

In [10]:
50# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 10

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(vocab_size=len(ids_from_chars.get_vocabulary()), embedding_dim=embedding_dim, rnn_units=rnn_units)
model.build((64, 200))
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  380       
_________________________________________________________________
gru_1 (GRU)                  multiple                  3182592   
_________________________________________________________________
dense_1 (Dense)              multiple                  38950     
Total params: 3,221,922
Trainable params: 3,221,922
Non-trainable params: 0
_________________________________________________________________


In [13]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

EPOCHS = 30

history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30


2021-11-18 15:36:40.370848: W tensorflow/core/common_runtime/bfc_allocator.cc:456] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.14MiB (rounded to 12730368)requested by op concat_0
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2021-11-18 15:36:40.370910: I tensorflow/core/common_runtime/bfc_allocator.cc:991] BFCAllocator dump for GPU_0_bfc
2021-11-18 15:36:40.370937: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (256): 	Total Chunks: 46, Chunks in use: 46. 11.5KiB allocated for chunks. 11.5KiB in use in bin. 1.1KiB client-requested in use in bin.
2021-11-18 15:36:40.370954: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2021-11-18 15:36:40.370971: I tensorflow/core/comm

CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node gradient_tape/my_model_1/embedding_1/embedding_lookup/Reshape/_18}}]] [Op:__inference_train_function_5068]

Function call stack:
train_function


by op Fill action_count 5697193716543326055 step 0 next 33
2021-11-18 15:36:40.371787: I tensorflow/core/common_runtime/bfc_allocator.cc:1046] InUse at 7f03c0080900 of size 155648 by op Fill action_count 5697193716543326057 step 0 next 20
2021-11-18 15:36:40.371804: I tensorflow/core/common_runtime/bfc_allocator.cc:1046] InUse at 7f03c00a6900 of size 155648 by op Add action_count 5697193716543326032 step 0 next 19
2021-11-18 15:36:40.371817: I tensorflow/core/common_runtime/bfc_allocator.cc:1046] InUse at 7f03c00cc900 of size 12582912 by op Fill action_count 5697193716543326053 step 0 next 31
2021-11-18 15:36:40.371829: I tensorflow/core/common_runtime/bfc_allocator.cc:1046] InUse at 7f03c0ccc900 of size 24698880 by op Fill action_count 5697193716543326058 step 0 next 12
2021-11-18 15:36:40.371842: I tensorflow/core/common_runtime/bfc_allocator.cc:1046] InUse at 7f03c245a900 of size 12582912 by op Mul action_count 5697193716543326019 step 0 next 14
2021-11-18 15:36:40.371853: I tensorf

In [12]:
#model.save_weights('pesos/embedings.hdf5')
#model.load_weights('pesos/embedings.hdf5')

ValueError: Cannot assign to variable embedding_1/embeddings:0 due to variable shape (38, 10) and value shape (38, 256) are incompatible

# Generando texto! 

In [1]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.6):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

NameError: name 'tf' is not defined

In [12]:
states = None
next_char = tf.constant(['amor '])
result = [next_char]

for n in range(500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

amor  la tierra que hace poco 
si no te metas te muero
y soy grande me ausente
de nuevo volvere

sembrando estrellas vivir

perdon por ti cantan alli 
los teje mas de una vez

a la huella  a la huella
solito  senior 
con esta zamba que en cada esquina del portezuelo

cuando se agranda la noche
en la primavera 
florecen los santiaguenios
donde estara la estrella que mate
el se le curan las primaveras

salgan mis saben si te van a decir
que el hombre que te estoy muy serio
y te amare que estoy viejo de 

________________________________________________________________________________
